# Change of Measure Theorem: Visualization with PyTorch

This notebook demonstrates the change of measure theorem for probability densities under monotonic transformations, using PyTorch for computation and Plotly for visualization. We use a sum of sigmoid functions to create a strictly increasing function, transform a standard normal density, and visualize the effect of the change of variable formula.

## 1. Import Required Libraries

Import torch, numpy, scipy.stats, and plotly for computation and visualization.

In [1]:
import torch
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

## 2. Define Sigmoid Function and Its Derivative in PyTorch

Implement the sigmoid function and its derivative using PyTorch tensors and autograd.

In [3]:
def sigmoid(x, loc, gain):
    return 1.0 / (1.0 + torch.exp(-(x - loc) / gain))


def sigmoid_sum(x, locs, gains):
    # x: [N], locs: [K], gains: [K]
    sigs = torch.stack([sigmoid(x, loc, gain) for loc, gain in zip(locs, gains)], dim=0)
    return sigs.sum(dim=0)


def sigmoid_sum_derivative(x, locs, gains):
    x = x.clone().detach().requires_grad_(True)
    y = sigmoid_sum(x, locs, gains)
    grad = torch.autograd.grad(
        y, x, grad_outputs=torch.ones_like(y), create_graph=False
    )[0]
    return grad.detach()

## 3. Set Parameters for Sigmoid Features

Define the locations and gains for three sigmoid functions as torch tensors, and create the domain for x.

In [18]:
# Parameters for three sigmoid features
locs = torch.tensor([-1.0, 0.0, 1.0])
gains = torch.tensor([1.0, 1.0, 1.0])

# Domain for x
N = 400
x = torch.linspace(-3, 3, N)

## 4. Compute and Visualize the Monotonic Function

Sum the three sigmoid functions to create a strictly increasing function $f(x)$ and plot the individual sigmoids and their derivatives using Plotly.

In [19]:
# Compute the sum of sigmoids and their derivatives
f = sigmoid_sum(x, locs, gains)
df = sigmoid_sum_derivative(x, locs, gains)


In [17]:
df

tensor([0.1678, 0.1699, 0.1720, 0.1741, 0.1763, 0.1784, 0.1806, 0.1828, 0.1851,
        0.1873, 0.1896, 0.1918, 0.1941, 0.1965, 0.1988, 0.2012, 0.2036, 0.2060,
        0.2084, 0.2108, 0.2133, 0.2158, 0.2183, 0.2208, 0.2233, 0.2259, 0.2285,
        0.2311, 0.2337, 0.2363, 0.2390, 0.2416, 0.2443, 0.2470, 0.2497, 0.2525,
        0.2553, 0.2580, 0.2608, 0.2636, 0.2665, 0.2693, 0.2722, 0.2751, 0.2780,
        0.2809, 0.2838, 0.2868, 0.2897, 0.2927, 0.2957, 0.2987, 0.3017, 0.3047,
        0.3078, 0.3108, 0.3139, 0.3170, 0.3201, 0.3232, 0.3263, 0.3294, 0.3325,
        0.3357, 0.3388, 0.3420, 0.3452, 0.3484, 0.3516, 0.3548, 0.3580, 0.3612,
        0.3644, 0.3676, 0.3709, 0.3741, 0.3773, 0.3806, 0.3838, 0.3871, 0.3903,
        0.3936, 0.3968, 0.4001, 0.4033, 0.4066, 0.4099, 0.4131, 0.4164, 0.4196,
        0.4229, 0.4261, 0.4293, 0.4326, 0.4358, 0.4390, 0.4423, 0.4455, 0.4487,
        0.4519, 0.4550, 0.4582, 0.4614, 0.4646, 0.4677, 0.4708, 0.4740, 0.4771,
        0.4802, 0.4832, 0.4863, 0.4894, 

In [16]:
x.grad

tensor([0.1678, 0.1699, 0.1720, 0.1741, 0.1763, 0.1784, 0.1806, 0.1828, 0.1851,
        0.1873, 0.1896, 0.1918, 0.1941, 0.1965, 0.1988, 0.2012, 0.2036, 0.2060,
        0.2084, 0.2108, 0.2133, 0.2158, 0.2183, 0.2208, 0.2233, 0.2259, 0.2285,
        0.2311, 0.2337, 0.2363, 0.2390, 0.2416, 0.2443, 0.2470, 0.2497, 0.2525,
        0.2553, 0.2580, 0.2608, 0.2636, 0.2665, 0.2693, 0.2722, 0.2751, 0.2780,
        0.2809, 0.2838, 0.2868, 0.2897, 0.2927, 0.2957, 0.2987, 0.3017, 0.3047,
        0.3078, 0.3108, 0.3139, 0.3170, 0.3201, 0.3232, 0.3263, 0.3294, 0.3325,
        0.3357, 0.3388, 0.3420, 0.3452, 0.3484, 0.3516, 0.3548, 0.3580, 0.3612,
        0.3644, 0.3676, 0.3709, 0.3741, 0.3773, 0.3806, 0.3838, 0.3871, 0.3903,
        0.3936, 0.3968, 0.4001, 0.4033, 0.4066, 0.4099, 0.4131, 0.4164, 0.4196,
        0.4229, 0.4261, 0.4293, 0.4326, 0.4358, 0.4390, 0.4423, 0.4455, 0.4487,
        0.4519, 0.4550, 0.4582, 0.4614, 0.4646, 0.4677, 0.4708, 0.4740, 0.4771,
        0.4802, 0.4832, 0.4863, 0.4894, 

In [12]:
fig = go.Figure()

# Plot individual sigmoids and their derivatives
for i, (loc, gain) in enumerate(zip(locs, gains)):
    sig = sigmoid(x, loc, gain).detach().numpy()
    # Derivative: s * (1 - s) / gain
    s = sig
    ds = s * (1 - s) / gain.item()
    fig.add_trace(
        go.Scatter(
            x=x,
            y=sig,
            mode="lines",
            line=dict(color="gray", width=1),
            name="sigmoid" if i == 0 else None,
            showlegend=(i == 0),
        )
    )
    fig.add_trace(
        go.Scatter(
            x=x,
            y=ds,
            mode="lines",
            line=dict(color="gray", width=1, dash="dash"),
            name="sigmoid derivative" if i == 0 else None,
            showlegend=(i == 0),
        )
    )

# Plot sum of sigmoids and its derivative
fig.add_trace(
    go.Scatter(
        x=x,
        y=f,
        mode="lines",
        line=dict(color="red", width=2),
        name="$f(x)$ (sum of sigmoids)",
    )
)
fig.add_trace(
    go.Scatter(
        x=x,
        y=df,
        mode="lines",
        line=dict(color="red", width=2, dash="dash"),
        name="$df/dx$",
    )
)

fig.update_layout(
    title="Monotonic Function $f(x)$ and Its Derivative",
    xaxis_title="$x$",
    yaxis_title="Value",
    legend=dict(itemsizing="constant"),
    width=1200,
    height=500,
)
fig.show()

## 5. Compute and Plot the Original Density $p_X(x)$

Compute the standard normal density using scipy.stats and plot it over the domain using Plotly.

In [44]:
# Standard normal density
p_x = torch.from_numpy(norm.pdf(x, loc=0, scale=1))

fig_px = go.Figure()
fig_px.add_trace(
    go.Scatter(
        x=x,
        y=p_x,
        mode="lines",
        line=dict(color="black"),
        name="$p_X(x)$ (standard normal)",
    )
)
fig_px.update_layout(
    title="Original Density $p_X(x)$",
    xaxis_title="$x$",
    yaxis_title="Density",
    legend=dict(itemsizing="constant"),
    width=1300,
    height=300,
)
fig_px.show()


## 6. Compute and Plot the Transformed Density $p_X(v(y))$

Numerically invert $f(x)$ to find $x = v(y)$ for each $y$, and plot the naive transformed density $p_X(v(y))$ as a function of $y$.

In [46]:
from scipy.interpolate import interp1d

# y values (sorted)
y = f
sort_idx = np.argsort(y)
y_sorted = y[sort_idx]
x_sorted = x[sort_idx]

# Inverse function: v(y)
v_func = interp1d(y_sorted, x_sorted, bounds_error=False, fill_value="extrapolate")

# Evaluate p_X at v(y)
p_x_vy = torch.from_numpy(norm.pdf(v_func(y_sorted), loc=0, scale=1))

fig_vy = go.Figure()
fig_vy.add_trace(
    go.Scatter(
        x=y_sorted,
        y=p_x_vy,
        mode="lines",
        line=dict(color="orange", dash="dashdot"),
        name="$p_X(v(y))$ (naive)",
    )
)
fig_vy.update_layout(
    title="Transformed Density $p_X(v(y))$ (without correction)",
    xaxis_title="$y$",
    yaxis_title="Density",
    legend=dict(itemsizing="constant"),
    width=900,
    height=350,
)
fig_vy.show()

## 7. Apply Change of Measure Correction and Plot $p_Y(y)$

Multiply $p_X(v(y))$ by the absolute value of the derivative of the inverse function to obtain the correct transformed density $p_Y(y)$ and plot it.

In [47]:
# Compute derivative of v(y): dv/dy = 1 / (df/dx) at x = v(y)
df_interp = interp1d(x, df, bounds_error=False, fill_value="extrapolate")
df_at_vy = df_interp(v_func(y_sorted))
dv_dy = 1.0 / df_at_vy

# Corrected density
p_y = p_x_vy * np.abs(dv_dy)

fig_py = go.Figure()
fig_py.add_trace(
    go.Scatter(
        x=y_sorted,
        y=p_x_vy,
        mode="lines",
        line=dict(color="orange", dash="dashdot"),
        name="$p_X(v(y))$ (naive)",
    )
)
fig_py.add_trace(
    go.Scatter(
        x=y_sorted,
        y=p_y,
        mode="lines",
        line=dict(color="green"),
        name="$p_Y(y)$ (corrected)",
    )
)
fig_py.update_layout(
    title="Transformed Density with Change of Measure Correction",
    xaxis_title="$y$",
    yaxis_title="Density",
    legend=dict(itemsizing="constant"),
    width=900,
    height=350,
)
fig_py.show()

/var/folders/0b/bk8cqvp552bglhb3xz88dsmh0000gp/T/ipykernel_19700/3855236318.py:7: DeprecationWarning:

__array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)



## 8. Compare Integrals of Densities

Numerically integrate $p_X(x)$, $p_X(v(y))$, and $p_Y(y)$ to verify normalization and demonstrate the effect of the correction.

In [48]:
from torch import trapz

int_px = trapz(p_x, x)
int_pxvy = trapz(p_x_vy, y_sorted)
int_py = trapz(p_y, y_sorted)

print(f"Integral of $p_X(x)$: {int_px:.4f}")
print(f"Integral of $p_X(v(y))$: {int_pxvy:.4f}")
print(f"Integral of $p_Y(y)$: {int_py:.4f}")

Integral of $p_X(x)$: 0.9973
Integral of $p_X(v(y))$: 0.5621
Integral of $p_Y(y)$: 0.9973


As expected, the integral of $p_X(x)$ and $p_Y(y)$ are both close to 1, while the naive transformed density $p_X(v(y))$ does not integrate to 1. This demonstrates the necessity of the change of measure correction.